# Rasterio

Rasterio reads and writes geospatial raster datasets.
https://travis-ci.org/mapbox/rasterio.png?branch=master https://coveralls.io/repos/github/mapbox/rasterio/badge.svg?branch=master

Rasterio employs GDAL under the hood for file I/O and raster formatting. Its functions typically accept and return Numpy ndarrays. Rasterio is designed to make working with geospatial raster data more productive and more fun.

Rasterio is pronounced raw-STEER-ee-oh.
## Example

Here's a simple example of the basic features rasterio provides. Three bands are read from an image and summed to produce something like a panchromatic band. This new band is then written to a new single band TIFF.

### Install
```
source activate GISpark
pip install rasterio
```

### 读取SRTM的单波段高程值。

In [14]:
import numpy
import rasterio
from pprint import *
# Read raster bands directly to Numpy arrays.
url = "../data/N00E018.hgt"
src = rasterio.open(url)
av = src.read()

pprint(src.profile)
print("ndArray.Shape:",av.shape)
pprint(av)

src.close()

{'affine': Affine(0.0008333333333333334, 0.0, 17.999583333333334,
       0.0, -0.0008333333333333334, 1.0004166666666667),
 'count': 1,
 'crs': {'init': 'epsg:4326'},
 'driver': 'SRTMHGT',
 'dtype': 'int16',
 'height': 1201,
 'nodata': -32768.0,
 'tiled': False,
 'transform': (17.999583333333334,
               0.0008333333333333334,
               0.0,
               1.0004166666666667,
               0.0,
               -0.0008333333333333334),
 'width': 1201}
ndArray.Shape: (1, 1201, 1201)
array([[[330, 326, 329, ..., 341, 340, 340],
        [331, 330, 327, ..., 340, 338, 339],
        [330, 330, 331, ..., 340, 342, 340],
        ..., 
        [323, 322, 324, ..., 331, 328, 330],
        [322, 323, 324, ..., 328, 328, 328],
        [321, 322, 323, ..., 331, 330, 332]]], dtype=int16)


In [27]:
print(av.max(),"~",av.min(),"==",av.mean())
av.view()

388 ~ -32768 == -66.3491587984


array([[[330, 326, 329, ..., 341, 340, 340],
        [331, 330, 327, ..., 340, 338, 339],
        [330, 330, 331, ..., 340, 342, 340],
        ..., 
        [323, 322, 324, ..., 331, 328, 330],
        [322, 323, 324, ..., 328, 328, 328],
        [321, 322, 323, ..., 331, 330, 332]]], dtype=int16)

### 读取影像的多波段值到多个变量。

In [1]:
# Read raster bands directly to Numpy arrays.
#
url = 'tests/data/RGB.byte.tif'
with rasterio.open(url) as src:
    r, g, b = src.read()

# Combine arrays in place. Expecting that the sum will
# temporarily exceed the 8-bit integer range, initialize it as
# a 64-bit float (the numpy default) array. Adding other
# arrays to it in-place converts those arrays "up" and
# preserves the type of the total array.
total = numpy.zeros(r.shape)
for band in r, g, b:
    total += band
total /= 3

# Write the product as a raster band to a new 8-bit file. For
# the new file's profile, we start with the meta attributes of
# the source file, but then change the band count to 1, set the
# dtype to uint8, and specify LZW compression.
profile = src.profile
profile.update(dtype=rasterio.uint8, count=1, compress='lzw')

with rasterio.open('example-total.tif', 'w', **profile) as dst:
    dst.write(total.astype(rasterio.uint8), 1)

ImportError: No module named 'rasterio'

The output:
http://farm6.staticflickr.com/5501/11393054644_74f54484d9_z_d.jpg

## API Overview

Simple access is provided to properties of a geospatial raster file.

In [ ]:
with rasterio.open('tests/data/RGB.byte.tif') as src:
    print(src.width, src.height)
    print(src.crs)
    print(src.affine)
    print(src.count)
    print(src.indexes)

# Printed:
# (791, 718)
# {u'units': u'm', u'no_defs': True, u'ellps': u'WGS84', u'proj': u'utm', u'zone': 18}
# Affine(300.0379266750948, 0.0, 101985.0,
#        0.0, -300.041782729805, 2826915.0)
# 3
# [1, 2, 3]

A dataset also provides methods for getting extended array slices given georeferenced coordinates and vice versa.

In [ ]:
with rasterio.open('tests/data/RGB.byte.tif') as src:
    print src.window(**src.window_bounds(((100, 200), (100, 200))))

# Printed:
# ((100, 200), (100, 200))

## Rasterio CLI

Rasterio's command line interface, named "rio", is documented at cli.rst. Its rio insp command opens the hood of any raster dataset so you can poke around using Python.

$ rio insp tests/data/RGB.byte.tif
Rasterio 0.10 Interactive Inspector (Python 3.4.1)
Type "src.meta", "src.read(1)", or "help(src)" for more information.
>>> src.name
'tests/data/RGB.byte.tif'
>>> src.closed
False
>>> src.shape
(718, 791)
>>> src.crs
{'init': 'epsg:32618'}
>>> b, g, r = src.read()
>>> b
masked_array(data =
 [[-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 ...,
 [-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]
 [-- -- -- ..., -- -- --]],
             mask =
 [[ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 ...,
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]
 [ True  True  True ...,  True  True  True]],
       fill_value = 0)

>>> b.min(), b.max(), b.mean()
(1, 255, 44.434478650699106)

## Rio Plugins

Rio provides the ability to create additional subcommands using plugins. See cli.rst for more information on building plugins.

See the plugin registry for a list of available plugins.

### Installation
#### Dependencies

Rasterio has one C library dependency: GDAL >=1.9. GDAL itself depends on a number of other libraries provided by most major operating systems and also depends on the non standard GEOS and PROJ4 libraries.

Python package dependencies (see also requirements.txt): affine, cligj (and click), enum34, numpy.

Development also requires (see requirements-dev.txt) Cython and other packages.
#### Installing from binaries
OS X

Binary wheels with the GDAL, GEOS, and PROJ4 libraries included are available for OS X versions 10.7+ starting with Rasterio version 0.17. To install, just run pip install rasterio. These binary wheels are preferred by newer versions of pip. If you don't want these wheels and want to install from a source distribution, run pip install rasterio --no-use-wheel instead.

The included GDAL library is fairly minimal, providing only the format drivers that ship with GDAL and are enabled by default. To get access to more formats, you must build from a source distribution (see below).

Binary wheels for other operating systems will be available in a future release.
Windows

Binary wheels for rasterio and GDAL are created by Christoph Gohlke and are available from his website.

To install rasterio, simply download both binaries for your system (rasterio and GDAL) and run something like this from the downloads folder:
```
$ pip install -U pip
$ pip install GDAL-1.11.2-cp27-none-win32.whl
$ pip install rasterio-0.24.0-cp27-none-win32.whl
```

#### Installing from the source distribution

Rasterio is a Python C extension and to build you'll need a working compiler (XCode on OS X etc). You'll also need Numpy preinstalled; the Numpy headers are required to run the rasterio setup script. Numpy has to be installed (via the indicated requirements file) before rasterio can be installed. See rasterio's Travis configuration for more guidance.

Linux

The following commands are adapted from Rasterio's Travis-CI configuration.
```
$ sudo add-apt-repository ppa:ubuntugis/ppa
$ sudo apt-get update
$ sudo apt-get install python-numpy libgdal1h gdal-bin libgdal-dev
$ pip install rasterio
```

Adapt them as necessary for your Linux system.

OS X

For a Homebrew based Python environment, do the following.

$ brew install gdal
$ pip install rasterio

Windows

You can download a binary distribution of GDAL from here. You will also need to download the compiled libraries and headers (include files).

When building from source on Windows, it is important to know that setup.py cannot rely on gdal-config, which is only present on UNIX systems, to discover the locations of header files and libraries that rasterio needs to compile its C extensions. On Windows, these paths need to be provided by the user. You will need to find the include files and the library files for gdal and use setup.py as follows.

$ python setup.py build_ext -I<path to gdal include files> -lgdal_i -L<path to gdal library>
$ python setup.py install

We have had success compiling code using the same version of Microsoft's Visual Studio used to compile the targeted version of Python (more info on versions used here.).

Note: The GDAL dll (gdal111.dll) and gdal-data directory need to be in your Windows PATH otherwise rasterio will fail to work.
Testing

From the repo directory, run py.test

$ python -m pytest

Note: some tests do not succeed on Windows (see #66.).
Documentation

See https://github.com/mapbox/rasterio/tree/master/docs.
License

See LICENSE.txt.
Authors

See AUTHORS.txt.
Changes

See CHANGES.txt.